# Complex Float16 in HDF5

Compare performance of complex float16 to float32 data type conversion using HDF5 and numpy.

In [1]:
import h5py
import numpy as np
print(h5py.version.info)

Summary of the h5py configuration
---------------------------------

h5py    3.7.0
HDF5    1.12.2
Python  3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]
sys.platform    darwin
sys.maxsize     9223372036854775807
numpy   1.23.3
cython (built with) 0.29.32
numpy (built against) 1.23.2
HDF5 (built against) 1.12.2



## Test Data

In [2]:
# Name of file
filename = "/tmp/test_complex.h5"
# Name of dataset in file
dataset_name = "complex_float16"
# Dimensions of output dataset
shape = 16384, 13440
# Standard deviation for normally distributed values,
# or `None` for random 16-bit patterns.
stdev = None
# Seed for random number generator (for reproducible data).
seed = 0
# Ensure no Inf or Nan values. 
disable_special = False
# Ensure no subnormal values (leading zeros in mantissa).
disable_subnormal = False

In [3]:
# Not natively supported in numpy, see
# https://github.com/numpy/numpy/issues/14753
complex32 = np.dtype([('r', np.float16), ('i', np.float16)])

In [4]:
rng = np.random.default_rng(seed)
if stdev is None:
    print("random 16-bit patterns")
    rand = lambda shape: rng.integers(0, 2**16, shape, np.uint16).view(np.float16)
else:
    print(f"float data in normal(0, {args.stdev}) distribution")
    rand = lambda shape: rng.normal(0.0, args.stdev, shape).astype(np.float16)
xf = np.zeros(shape, complex32)
xf['r'] = rand(shape)
xf['i'] = rand(shape)

if disable_special:
    print("setting values in {inf, -inf, nan} to zero")
    invalid = np.isinf(xf['r']) | np.isnan(xf['r']) | np.isinf(xf['i']) | np.isnan(xf['i'])
    xf[invalid] = 0.0

if disable_subnormal:
    print("setting subnormal values to zero")
    fp = np.finfo(xf['r'].dtype)
    invalid = (np.abs(xf['r']) < fp.tiny) | (np.abs(xf['i']) < fp.tiny)
    xf[invalid] = 0.0

print(f"creating file {filename}")
with h5py.File(filename, "w") as h5:
    print(f"writing to dataset {dataset_name}")
    dset = h5.create_dataset(dataset_name, data=xf)

random 16-bit patterns
creating file /tmp/test_complex.h5
writing to dataset complex_float16


In [5]:
f = h5py.File(filename, "r")
dset = f[dataset_name]

## Convert with Numpy

In [6]:
complex64 = np.dtype([('r', np.float32), ('i', np.float32)])

In [7]:
%%time
# Use HDF5 API to read float16 data
# Need to work around h5py bug, see
# https://github.com/h5py/h5py/issues/2156
x = dset.astype(complex32)[:]
# Use Numpy API to convert float16 to float32
y = x.astype(complex64).view(np.complex64)[:]

CPU times: user 1.32 s, sys: 660 ms, total: 1.98 s
Wall time: 1.98 s


## Convert with HDF5

In [8]:
%%time
# Use HDF5 API to read data converted to float32
# Note h5py API changed API for this in v3.0.0 (old way used a context manager)
z = dset.astype(np.complex64)[:]

CPU times: user 1min 30s, sys: 446 ms, total: 1min 30s
Wall time: 1min 30s


The HDF5 version is about 45x slower than numpy (90 s versus 2 s on my laptop).

## Compare Results

In [9]:
# Verify that both methods decode to same values.
# Just use real part.
mismatch = y.real != z.real
if not np.alltrue(~mismatch):
    i, j = np.where(mismatch)
    print(f"mismatch at {len(i)} values")
    if np.all(np.isnan(xf['r'][mismatch])):
        print("all mismatched values are nan")
    for k in range(min(5, len(i))):
        key = i[k], j[k]
        print(f"example value at index {key}:")
        print("  input float16:     ", xf['r'][key])
        print("  input float16 bits:", bin(xf['r'][key].view(np.uint16)))
        print("  numpy float32 bits:", bin(y.real[key].view(np.uint32)))
        print("  HDF5  float32 bits:", bin(z.real[key].view(np.uint32)))

mismatch at 6875250 values
all mismatched values are nan
example value at index (0, 32):
  input float16:      nan
  input float16 bits: 0b111111110111111
  numpy float32 bits: 0b1111111111101111110000000000000
  HDF5  float32 bits: 0b1111111111111111111111111111111
example value at index (0, 64):
  input float16:      nan
  input float16 bits: 0b111111100001110
  numpy float32 bits: 0b1111111111000011100000000000000
  HDF5  float32 bits: 0b1111111111111111111111111111111
example value at index (0, 107):
  input float16:      nan
  input float16 bits: 0b1111111101001001
  numpy float32 bits: 0b11111111111010010010000000000000
  HDF5  float32 bits: 0b11111111111111111111111111111111
example value at index (0, 118):
  input float16:      nan
  input float16 bits: 0b111111111000110
  numpy float32 bits: 0b1111111111110001100000000000000
  HDF5  float32 bits: 0b1111111111111111111111111111111
example value at index (0, 147):
  input float16:      nan
  input float16 bits: 0b111110001011111

The outputs are identical except for NaN values, where HDF5 seems to just set all the significand to one. This is fairly harmless but could be annoying if the NaNs contain a payload for debugging.